In [ ]:
# %% ----------------  burn media.png onto the recorded screen  ----------------
import cv2, json, numpy as np, logging, pathlib

JSON_TRACK = "aruco_screen_tracks.json"  # from previous export
VIDEO_IN = "aruco.mp4"
MEDIA_IMG = "media.png"
VIDEO_OUT = "screen_with_media.mp4"
ALPHA = 0.0  # set >0 if you want the video faintly visible behind media

# ─── tiny log (optional) ─────────────────────────────────────────────────────
logging.basicConfig(
    filename="replace_debug.log",
    filemode="w",
    level=logging.DEBUG,
    format="%(message)s",
)
log = logging.getLogger("replace")

# ─── load JSON ───────────────────────────────────────────────────────────────
with open(JSON_TRACK, "r", encoding="utf-8") as f:
    info = json.load(f)

frames_info = info["frames"]
FW, FH = info["frame_width"], info["frame_height"]

# ─── open video + media -------------------------------------------------------
cap = cv2.VideoCapture(VIDEO_IN)
fps = cap.get(cv2.CAP_PROP_FPS)
four = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(VIDEO_OUT, four, fps, (FW, FH))

media = cv2.imread(MEDIA_IMG, cv2.IMREAD_COLOR)
if media is None:
    raise RuntimeError(f"Cannot read {MEDIA_IMG}")
mh, mw = media.shape[:2]

# source quad placeholder — will be updated per frame
src_quad = np.array([[0, 0], [1, 0], [1, 1], [0, 1]], np.float32)

for idx, rec in enumerate(frames_info):
    ok, frame = cap.read()
    if not ok:
        break

    if not rec["valid"]:
        out.write(frame)  # no overlay this frame
        continue

    dst_quad = np.array(rec["corners"], np.float32)  # TL,TR,BR,BL

    # ---- overlay size in this frame ---------------------------------------
    ov_w = int(np.linalg.norm(dst_quad[1] - dst_quad[0]) + 0.5)  # top edge
    ov_h = int(np.linalg.norm(dst_quad[3] - dst_quad[0]) + 0.5)  # left edge

    # ---- first try: scale by width ----------------------------------------
    scale_w = ov_w / mw
    tmp = cv2.resize(media, (ov_w, int(mh * scale_w)), interpolation=cv2.INTER_AREA)
    th, tw = tmp.shape[:2]  # tw == ov_w

    if th >= ov_h:  # tall enough → crop/keep top
        patch = tmp[:ov_h, :]  # cut bottom if needed
    else:
        # ---- second try: scale by height; then center-crop width ----------
        scale_h = ov_h / mh
        tmp = cv2.resize(media, (int(mw * scale_h), ov_h), interpolation=cv2.INTER_AREA)
        th, tw = tmp.shape[:2]  # th == ov_h
        if tw > ov_w:  # crop equally from both sides
            x0 = (tw - ov_w) // 2
            patch = tmp[:, x0 : x0 + ov_w]
        else:
            patch = tmp  # rare corner-case: exact fit

    # src quad now exactly matches patch size
    ph, pw = patch.shape[:2]  # pw == ov_w , ph == ov_h
    src_quad[:] = [[0, 0], [pw - 1, 0], [pw - 1, ph - 1], [0, ph - 1]]

    # ---- warp & composite --------------------------------------------------
    H = cv2.getPerspectiveTransform(src_quad, dst_quad)
    warped = cv2.warpPerspective(patch, H, (FW, FH))

    mask = cv2.warpPerspective(np.ones((ph, pw), np.uint8) * 255, H, (FW, FH))
    if ALPHA > 0:
        bg = cv2.addWeighted(frame, 1 - ALPHA, warped, ALPHA, 0, mask=None)
    else:
        inv = cv2.bitwise_not(mask)
        bg = cv2.bitwise_and(frame, frame, mask=inv)
    final = cv2.bitwise_or(bg, cv2.bitwise_and(warped, warped, mask=mask))

    out.write(final)

cap.release()
out.release()
print("✅  Done – output saved to", VIDEO_OUT)


✅  Done – output saved to screen_with_media.mp4
